In [35]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=6, app_name='west0', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 6 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [44]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [32]:
device = "console"

#16.1 update  
PC: 2/16 ~ 3/1  
CONSOLE: 2/24 ~ 3/9

In [7]:
# Day 10
oct_start_date = "2021-10-14"
oct_end_date = "2021-10-23"
nov_start_date = "2021-11-11"
nov_end_date = "2021-11-20"
dec_start_date = "2021-12-09"
dec_end_date = "2021-12-18"

feb_start_date = "2022-02-24"
feb_end_date = "2022-03-05"
march_start_date = "2022-03-24"
march_end_date = "2022-04-03"

In [37]:
# Day 28
oct_start_date = "2021-10-14"
oct_end_date = "2021-11-10"
nov_start_date = "2021-11-11"
nov_end_date = "2021-12-08"
dec_start_date = "2021-12-09"
dec_end_date = "2022-01-05"

feb_start_date = "2022-02-24"
feb_end_date = "2022-03-23"
# march_start_date = "2022-03-24"
# march_end_date = "2022-04-03"

In [38]:
gcoin10 = load_data_mart(device, oct_start_date, oct_end_date, "gcoin_use")
gcoin11 = load_data_mart(device, nov_start_date, nov_end_date, "gcoin_use")
gcoin12 = load_data_mart(device, dec_start_date, dec_end_date, "gcoin_use")
gcoin2 = load_data_mart(device, feb_start_date, feb_end_date, "gcoin_use")
# gcoin3 = load_data_mart(device, march_start_date, march_end_date, "gcoin_use")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46023)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46023)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

### WSUS

In [40]:
gcoin10_wsus = gcoin10.where(col("event_type") == "wsus")
gcoin11_wsus = gcoin11.where(col("event_type") == "wsus")
gcoin12_wsus = gcoin12.where(col("event_type") == "wsus")
gcoin2_wsus = gcoin2.where(col("event_type") == "wsus")
# gcoin3_wsus = gcoin3.where(col("event_type") == "wsus")

In [26]:
gcoin = gcoin10_wsus.unionByName(gcoin11_wsus)
gcoin = gcoin.unionByName(gcoin12_wsus)
gcoin = gcoin.unionByName(gcoin2_wsus)
gcoin = gcoin.unionByName(gcoin3_wsus)

In [27]:
gcoin.where(col("paid_use") > 0).select(sum("paid_use")/sum(col("paid_use") + col("free_use"))).show()

+--------------------------------------------+
|(sum(paid_use) / sum((paid_use + free_use)))|
+--------------------------------------------+
|                          0.8886461554694154|
+--------------------------------------------+



In [10]:
tmp10 = gcoin10_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("october"))
tmp11 = gcoin11_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("november"))
tmp12 = gcoin12_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("december"))
tmp2 = gcoin2_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("feburary"))
tmp3 = gcoin3_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")). \
    groupBy("price", "buy_cnt_group").agg(count("*").alias("user_cnt")). \
    withColumn("month", lit("march"))

In [11]:
wsus_by_month = tmp10.unionByName(tmp11)
wsus_by_month = wsus_by_month.unionByName(tmp12)
wsus_by_month = wsus_by_month.unionByName(tmp2)
wsus_by_month = wsus_by_month.unionByName(tmp3)

In [12]:
wsus_by_month_df = wsus_by_month.toPandas()

In [118]:
# wsus_by_month_df.to_csv("../Data/wsus_by_month_df.csv", index=False)

In [13]:
wsus_by_month_df.groupby(["price", "buy_cnt_group"])["user_cnt"].describe()

count     mean           std     min     25%     50%  \
price buy_cnt_group                                                         
200   None             5.0  11118.8  12067.587277  2794.0  3085.0  3254.0   
      multiple         5.0   3340.0   1652.829241  1394.0  2237.0  3534.0   
      one              5.0   2772.6   1150.775087  1164.0  2002.0  3267.0   
1800  None             5.0    892.4    874.228117    44.0   550.0   578.0   
      multiple         5.0   2443.0    729.589611  1666.0  2182.0  2191.0   
      one              5.0   2391.4   1228.954352  1115.0  1588.0  2382.0   

                         75%      max  
price buy_cnt_group                    
200   None           16396.0  30065.0  
      multiple        3816.0   5719.0  
      one             3497.0   3933.0  
1800  None             942.0   2348.0  
      multiple        2558.0   3618.0  
      one             2546.0   4326.0

In [41]:
wsus10 = gcoin10_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("october"))
wsus11 = gcoin11_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("november"))
wsus12 = gcoin12_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("december"))
wsus2 = gcoin2_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
    withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("feburary"))
# wsus3 = gcoin3_wsus.groupBy("account_id", "price").agg(sum(when(col("paid_use") > 0, 1)).alias("buy_cnt")). \
#     withColumn("buy_cnt_group", when(col("buy_cnt") == 1, "one").when(col("buy_cnt") > 1, "multiple").otherwise("None")).withColumn("month", lit("march"))

In [42]:
wsus_account = wsus10.unionByName(wsus11)
wsus_account = wsus_account.unionByName(wsus12)
wsus_account = wsus_account.unionByName(wsus2)
# wsus_account = wsus_account.unionByName(wsus3)

In [16]:
wsus_account.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- price: long (nullable = true)
 |-- buy_cnt: long (nullable = true)
 |-- buy_cnt_group: string (nullable = false)
 |-- month: string (nullable = false)



In [43]:
wsus_account.groupBy("month", "price", "buy_cnt_group").agg(sum("buy_cnt").alias("total_buy_cnt")).toPandas().groupby(["price", "buy_cnt_group"])["total_buy_cnt"].describe()

count      mean           std      min       25%  \
price buy_cnt_group                                                     
200   None             0.0       NaN           NaN      NaN       NaN   
      multiple         4.0  31117.75  12936.321847  15094.0  24364.00   
      one              4.0   4735.50   1257.282122   2998.0   4220.50   
1800  None             0.0       NaN           NaN      NaN       NaN   
      multiple         4.0  24585.00   7367.825188  18279.0  19068.00   
      one              4.0   3844.25   1914.065372   2277.0   2837.25   

                         50%       75%      max  
price buy_cnt_group                              
200   None               NaN       NaN      NaN  
      multiple       32026.5  38780.25  45324.0  
      one             5108.0   5623.00   5728.0  
1800  None               NaN       NaN      NaN  
      multiple       22955.0  28472.00  34151.0  
      one             3239.5   4246.50   6621.0

### Buy Count by Price Monthly

In [18]:
tmp10_price = gcoin10.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("october"))
tmp11_price = gcoin11.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("november"))
tmp12_price = gcoin12.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("december"))
tmp2_price = gcoin2.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("feburary"))
tmp3_price = gcoin3.groupBy("price").agg(sum(when(col("paid_use") > 0, 1)).alias("paid_buy_cnt"), count("*").alias("total_buy_cnt")).withColumn("month", lit("march"))

In [19]:
price_by_month = tmp10_price.unionByName(tmp11_price)
price_by_month = price_by_month.unionByName(tmp12_price)
price_by_month = price_by_month.unionByName(tmp2_price)
price_by_month = price_by_month.unionByName(tmp3_price)

In [20]:
price_by_month_df = price_by_month.toPandas()
# price_by_month_df.to_csv("../Data/price_by_month_df.csv", index=False)

In [21]:
price_by_month_df[(price_by_month_df.price >= 100) & (price_by_month_df.price <= 250)].groupby("price", as_index=False)["paid_buy_cnt"].mean()

,price,paid_buy_cnt
0,100,1511.000000
1,120,58.000000
2,140,37.000000
3,150,247.000000
4,170,13.666667
5,180,26.500000
6,200,19979.200000
7,210,34.333333
8,240,36.333333
9,250,97.000000


In [24]:
price_by_month_df[(price_by_month_df.price >= 900) & (price_by_month_df.price <= 1800)].groupby("price", as_index=False)["paid_buy_cnt"].mean()

,price,paid_buy_cnt
0,900,28.600000
1,910,32.500000
2,920,48.000000
3,940,6.000000
4,950,98.333333
5,960,15.000000
6,970,3.666667
7,980,4.333333
8,990,9495.600000
9,1000,490.800000


In [23]:
price_by_month_df[(price_by_month_df.price >= 1800) & (price_by_month_df.price <= 2500)].groupby("price", as_index=False)["paid_buy_cnt"].mean()

,price,paid_buy_cnt
0,1800,15678.400000
1,1880,2.666667
2,1980,139.200000
3,1990,204.600000
4,2000,63.333333
5,2070,6.000000
6,2080,1153.000000
7,2100,2.500000
8,2370,2.000000
9,2420,1.000000


In [14]:
feb_membership = load_membership("pubg", "pc", "2022-02-16", "2022-03-01").select("date", "account_id", "membership").distinct()

In [15]:
feb_gcoin_pc = feb_gcoin_pc.join(feb_membership, ["date", "account_id"], "left")

In [29]:
feb_gcoin_pc_tmp = feb_gcoin_pc.groupBy("date", "product_id", "product_name", "sales_id", "price", "membership").agg(countDistinct("account_id").alias("pu"), \
                                                                                       countDistinct(when(col("paid_use")>0, col("account_id")).otherwise(0)).alias("paid_pu"), \
                                                                                       sum("free_use").alias("free_use"), \
                                                                                       sum("paid_use").alias("paid_use")
                                                                                      )

In [32]:
mysql.insert_table(feb_gcoin_pc_tmp, "labs", "feb_gcoin_pc")

In [60]:
march_gcoin_pc = load_data_mart("pc", "2022-03-16", "2022-03-29", "gcoin_use")
march_membership = load_membership("pubg", "pc", "2022-03-16", "2022-03-29").select("date", "account_id", "membership").distinct()
march_gcoin_pc = march_gcoin_pc.join(march_membership, ["date", "account_id"], "left")

In [61]:
march_gcoin_pc_tmp = march_gcoin_pc.groupBy("date", "product_id", "product_name", "sales_id", "price", "membership").agg(countDistinct("account_id").alias("pu"), \
                                                                                       countDistinct(when(col("paid_use")>0, col("account_id")).otherwise(0)).alias("paid_pu"), \
                                                                                       sum("free_use").alias("free_use"), \
                                                                                       sum("paid_use").alias("paid_use")
                                                                                      )

In [62]:
mysql.insert_table(march_gcoin_pc_tmp, "labs", "march_gcoin_pc")

In [31]:
# mysql.drop_table("labs", "feb_gcoin_pc")

In [35]:
feb_wsus = feb_gcoin_pc.where(col("event_type") == "wsus")

In [38]:
feb_wsus.select("event_name").distinct().toPandas()

,event_name
0,202201_wsus
1,202111_wsus
2,202112_wsus_progressive
3,202202_wsus_progressive


In [40]:
feb_wsus_cnt_daily = feb_wsus.groupBy("date", "account_id", "membership", "price").agg(count("*").alias("buy_cnt"), sum(when(col("paid_use") > 0, 1).otherwise(0)).alias("paid_buy_cnt"))

In [44]:
# feb_wsus_cnt_daily_df = feb_wsus_cnt_daily.toPandas()
feb_wsus_cnt_daily_df.to_csv("../Data/feb_wsus_cnt_daily_df.csv", index=False)

In [45]:
feb_wsus_cnt_daily_df.head()

,date,account_id,membership,price,buy_cnt,paid_buy_cnt
0,2022-02-16,account.00ce68eee960409a8deaf46178fc94a5,legacy,200,3,0
1,2022-02-16,account.014d901691864512be0fe258c1ec38ef,legacy,200,1,0
2,2022-02-16,account.02277864522d4e7ead16316efdc90d58,legacy,200,6,0
3,2022-02-16,account.022c73a0054f44b5800372a994969b58,basic,200,2,0
4,2022-02-16,account.0247299a07614ed89f0073fd64e7964d,legacy,200,1,1


In [49]:
feb_wsus_cnt_daily_df.groupby(["date", "membership", "price"]).describe().reset_index().to_csv("../Data/feb_wsus_daily_describe.csv", index=False)

In [58]:
# 상자 오픈 1회
feb_wsus_cnt_daily_df[(feb_wsus_cnt_daily_df.price == 200) & (feb_wsus_cnt_daily_df.buy_cnt == 1)].groupby("date", as_index=False)[["buy_cnt", "paid_buy_cnt"]].sum()

,date,buy_cnt,paid_buy_cnt
0,2022-02-16,38814,8282
1,2022-02-17,39363,6183
2,2022-02-18,34350,4822
3,2022-02-19,36284,4340
4,2022-02-20,34808,3820
5,2022-02-21,15194,2212
6,2022-02-22,14466,2035
7,2022-02-23,17650,4784
8,2022-02-24,15992,3582
9,2022-02-25,15566,3153


In [54]:
feb_wsus_cnt_daily_df.groupby(["date", "price"]).describe()

buy_cnt                                                 \
                    count      mean       std  min  25%  50%  75%    max   
date       price                                                           
2022-02-16 200    88076.0  2.844475  3.732719  1.0  1.0  2.0  3.0  187.0   
           1800   36037.0  4.468685  8.985432  1.0  1.0  1.0  4.0  141.0   
2022-02-17 200    87500.0  2.702994  3.325971  1.0  1.0  2.0  3.0  303.0   
           1800   27900.0  3.507849  7.338281  1.0  1.0  1.0  3.0  124.0   
2022-02-18 200    77035.0  2.695632  3.036430  1.0  1.0  2.0  3.0  179.0   
           1800   23806.0  3.013862  6.151970  1.0  1.0  1.0  2.0  116.0   
2022-02-19 200    81589.0  2.688990  2.929741  1.0  1.0  2.0  3.0  176.0   
           1800   24091.0  2.646964  5.589806  1.0  1.0  1.0  2.0  122.0   
2022-02-20 200    77378.0  2.640983  2.801054  1.0  1.0  2.0  3.0  176.0   
           1800   21845.0  2.463493  5.277816  1.0  1.0  1.0  2.0  124.0   
2022-02-21 200    35649.0  2.728744  2.874092  1.0  1.0  2.0  3.0   88.0   
           1800   12396.0  2.830187  6.195572  1.0  1.0  1.0  2.0  128.0   
2022-02-22 200    34545.0  2.717180  2.803678  1.0  1.0  2.0  3.0  102.0   
           1800   11486.0  2.778078  6.151358  1.0  1.0  1.0  2.0  120.0   
2022-02-23 200    44213.0  2.655870  2.868570  1.0  1.0  2.0  3.0  144.0   
           1800   13979.0  2.610988  5.317455  1.0  1.0  1.0  2.0  112.0   
2022-02-24 200    38570.0  2.620119  2.686001  1.0  1.0  2.0  3.0   88.0   
           1800   11714.0  2.647345  5.307920  1.0  1.0  1.0  2.0  122.0   
2022-02-25 200    37303.0  2.649385  3.172744  1.0  1.0  2.0  3.0  248.0   
           1800   11868.0  2.770391  6.040838  1.0  1.0  1.0  2.0  111.0   
2022-02-26 200    40808.0  2.630612  2.765014  1.0  1.0  2.0  3.0  134.0   
           1800   12869.0  2.904577  6.511741  1.0  1.0  1.0  2.0  105.0   
2022-02-27 200    38884.0  2.674288  2.852493  1.0  1.0  2.0  3.0  108.0   
           1800   12073.0  3.050443  7.199619  1.0  1.0  1.0  2.0  106.0   
2022-02-28 200    31016.0  2.655307  2.723758  1.0  1.0  2.0  3.0   60.0   
           1800    9954.0  3.047016  7.065211  1.0  1.0  1.0  2.0  122.0   
2022-03-01 200    31766.0  2.622615  2.716652  1.0  1.0  2.0  3.0   95.0   
           1800   10106.0  2.799426  6.136534  1.0  1.0  1.0  2.0  110.0   

                 paid_buy_cnt                                                 
                        count      mean       std  min  25%  50%  75%    max  
date       price                                                              
2022-02-16 200        88076.0  1.045949  3.291019  0.0  0.0  0.0  1.0  181.0  
           1800       36037.0  4.049144  8.678071  0.0  0.0  1.0  4.0  115.0  
2022-02-17 200        87500.0  0.741771  2.749882  0.0  0.0  0.0  0.0  262.0  
           1800       27900.0  3.027778  7.147801  0.0  0.0  1.0  3.0  116.0  
2022-02-18 200        77035.0  0.638100  2.399961  0.0  0.0  0.0  0.0  145.0  
           1800       23806.0  2.513946  6.041566  0.0  0.0  1.0  2.0  108.0  
2022-02-19 200        81589.0  0.549829  2.244566  0.0  0.0  0.0  0.0  151.0  
           1800       24091.0  2.099041  5.526838  0.0  0.0  1.0  2.0  111.0  
2022-02-20 200        77378.0  0.484414  2.082843  0.0  0.0  0.0  0.0  170.0  
           1800       21845.0  1.886290  5.229496  0.0  0.0  0.0  1.0  117.0  
2022-02-21 200        35649.0  0.676793  2.319578  0.0  0.0  0.0  0.0   82.0  
           1800       12396.0  2.305663  6.095825  0.0  0.0  1.0  2.0  119.0  
2022-02-22 200        34545.0  0.634650  2.200717  0.0  0.0  0.0  0.0   91.0  
           1800       11486.0  2.221574  6.044827  0.0  0.0  1.0  2.0  116.0  
2022-02-23 200        44213.0  0.855608  2.423246  0.0  0.0  0.0  1.0  144.0  
           1800       13979.0  2.170255  5.284419  0.0  0.0  1.0  2.0  108.0  
2022-02-24 200        38570.0  0.753383  2.201356  0.0  0.0  0.0  1.0   82.0  
           1800       11714.0  2.177736  5.249288  0.0  0.0  1.0  2.0  111.0  
20

## 3월 BM별 카니발라이즈

In [64]:
march_bm = pd.read_csv("../Data/march_carnival.csv")

In [66]:
march_bm.columns = ["bm", "name", "date", "value"]

In [67]:
march_bm.dtypes

bm        object
name      object
date      object
value    float64
dtype: object

In [69]:
march_sales = march_bm[march_bm.name == "Paid GCoin Revenue"]

In [82]:
march_percent = march_bm[march_bm.name != "Paid GCoin Revenue"]

In [83]:
march_percent["value"] = march_percent["value"] * 100

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [84]:
march_percent.groupby(["date", "bm"], as_index=False)["value"].sum().pivot(index="date", columns="bm")

value                                                   
bm              esport ingame_items survivor_pass (+levels_up)       wsus
date                                                                     
2022년 3월 16일  6.749736    51.695374                  23.668358  17.886532
2022년 3월 17일  5.000868    53.182361                  18.853316  22.963455
2022년 3월 18일  4.135963    54.442382                  15.086021  26.335633
2022년 3월 19일  3.640249    55.100241                  13.890562  27.368947
2022년 3월 20일  3.338947    55.908356                  12.411723  28.340974
2022년 3월 21일  2.715737    56.081625                  10.885168  30.317470
2022년 3월 22일  2.520000    56.181885                   9.740076  31.558039
2022년 3월 23일  2.337389    56.650404                   9.085129  31.927078
2022년 3월 24일  2.121052    56.892521                   8.835904  32.150524
2022년 3월 25일  2.170793    56.829551                   8.422381  32.577275
2022년 3월 26일  1.783088    56.363115                   7.571692  34.282105
2022년 3월 27일  1.849191    56.893553                   7.799984  33.457272
2022년 3월 28일  1.543490    56.193804                   6.331690  35.931016
2022년 3월 29일  1.559627    56.305840                   6.271429  35.863105

In [73]:
march_sales.groupby(["date", "bm"], as_index=False)["value"].sum().pivot(index="date", columns="bm")

value                                                   
bm               esport ingame_items survivor_pass (+levels_up)       wsus
date                                                                      
2022년 3월 16일  376047.91   2880103.50                 1318634.84  996512.05
2022년 3월 17일  138954.95   1477733.85                  523861.35  638066.35
2022년 3월 18일   77759.20   1023557.40                  283628.45  495129.55
2022년 3월 19일   56396.00    853632.00                  215197.40  424009.20
2022년 3월 20일   43798.80    733380.65                  162811.40  371764.15
2022년 3월 21일   26938.90    556304.65                  107976.00  300735.75
2022년 3월 22일   22763.70    507503.05                   87984.20  285070.55
2022년 3월 23일   22749.50    551370.85                   88424.35  310742.00
2022년 3월 24일   18560.40    497841.70                   77319.15  281335.25
2022년 3월 25일   20894.10    546990.05                   81066.25  313559.50
2022년 3월 26일   20115.00    635831.85                   85416.20  386736.15
2022년 3월 27일   18111.20    557222.40                   76394.00  327684.60
2022년 3월 28일   14008.20    509996.20                   57464.30  326097.90
2022년 3월 29일   13418.10    484422.00                   53955.65  308544.85

In [75]:
march_sales.groupby(["bm"], as_index=False)["value"].sum()

,bm,value
0,esport,870515.96
1,ingame_items,11815890.15
2,survivor_pass (+levels_up),3220133.54
3,wsus,5765987.85
